<a href="https://colab.research.google.com/github/PancrePal-xiaoyibao/gemini2.0-xiaoyibao/blob/master/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [1]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-2.0-flash-exp' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

[
    {
        "role": "user",
        "parts": [
            {
                "text": "\u54a8\u8be2\u4e0b\u4e13\u5bb6\uff0c\u6211\u9700\u8981\u5f00\u53d1\u4e00\u4e2a\u672c\u5730\u90e8\u7f72\u7684\u60a3\u8005\u95ee\u7b54\u5e94\u7528\u5c0f\u7a0b\u5e8f, \u540e\u7aef\u5f00\u53d1\u7684\u4e3b\u8981\u5de5\u4f5c\u5305\u62ec\u54ea\u4e9b\uff0c\u524d\u7aef\u5f00\u53d1\u5de5\u4f5c\u6709\u54ea\u4e9b\uff0c\u8fd8\u6709\u4e2a\u95ee\u9898\uff0c\u4f7f\u7528api\u8c03\u7528rag\u670d\u52a1\uff0c\u8fd8\u662f\u4f7f\u7528web-view\u5d4c\u5165\u4e00\u4e2a\u5f00\u53d1\u597d\u7684\u4e5frag\u5bf9\u8bdd\u5e94\u7528\u9875\u9762\u66f4\u597d\uff1f\\n"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
                "text": "\u4f60\u597d\uff0c\u6211\u662f\u5c0f\u7a0b\u5e8f\u5f00\u53d1\u4e13\u5bb6 AK\u3002\u5f88\u9ad8\u5174\u80fd\u4e3a\u4f60\u89e3\u7b54\u5173\u4e8e\u672c\u5730\u90e8\u7f72\u60a3\u8005\u95ee\u7b54\u5c0f\u7a0b\u5e8f\u7684\u95ee\u9898\u3002 \\n\\n\u9996\u5148

## Call `generate_content`

In [2]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

\n**以下是我为你整理的当前项目状态，方便你快速回顾：**\n\n*   **已完成：**\n    *   明确了小程序的整体功能和业务需求。\n    *   确定了前后端技术栈和工具。\n    *   设计了详细的数据库表结构 (包括分表策略，数据类型和索引)。\n    *   设计了完整的 API 接口 (包括请求参数，响应数据，认证授权，错误处理)。\n    *   搭建了后端项目基本框架 (使用 Windsurf 生成)。\n    *   创建了前端项目基本框架 ( 使用 Cursor 或 VS Code 创建基本目录)。\n*   **待完成：**\n    *   根据数据库表结构，编写后端 `models` 代码。\n    *   编写后端 API 接口代码，并集成 RAG 平台。\n    *   编写单元测试。\n    *   部署后端服务。\n    *   根据 API 设计，编写小程序端代码。\n    *   前端页面，组件设计。\n    *   测试和部署小程序。\n\n**请记住，我在这里随时待命，为你提供支持！**


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [3]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")

Role: user 

咨询下专家，我需要开发一个本地部署的患者问答应用小程序, 后端开发的主要工作包括哪些，前端开发工作有哪些，还有个问题，使用api调用rag服务，还是使用web-view嵌入一个开发好的也rag对话应用页面更好？\n 

-------------------------------------------------------------------------------- 

Role: model 

你好，我是小程序开发专家 AK。很高兴能为你解答关于本地部署患者问答小程序的问题。 \n\n首先，我们来梳理一下这个项目的整体架构，以及前后端各自的职责，最后再讨论RAG服务集成方案的选择。\n\n**项目整体架构**\n\n这个本地部署的患者问答小程序，主要包含以下几个核心部分：\n\n1.  **小程序前端 (Client):** 用户交互界面，负责展示问题、答案，以及用户输入等功能。\n2.  **小程序后端 (Server):** 处理用户请求，与数据库交互，调用RAG服务，返回数据给前端。\n3.  **本地数据库 (Database):** 存储患者数据、知识库等。\n4.  **RAG 服务 (Retrieval-Augmented Generation):** 基于知识库，生成高质量的问答结果。\n\n**后端开发主要工作 (Server-Side)**\n\n1.  **API 设计与开发:**\n    *   **用户认证与授权:** 实现用户登录、注册、权限管理等功能。\n    *   **问题提交接口:** 接收用户提出的问题，并进行处理。\n    *   **答案返回接口:** 将 RAG 服务生成的答案返回给前端。\n    *   **数据管理接口:** 提供对患者数据、知识库的增删改查接口。\n    *   **数据同步接口:** (如果需要) 提供与其他系统的数据同步接口。\n\n2.  **RAG 服务集成:**\n    *   **RAG 服务部署:** 将 RAG 服务部署到本地服务器或指定环境。\n    *   **RAG 服务调用:** 实现后端代码与 RAG 服务的交互，传递用户问题，获取答案。\n    *   **RAG 服务配置:** 配置 RAG 服务的知识库，并根据需